In [1]:
import os
import numpy as np
import transformers
from datasets import Dataset, DatasetDict
from eval import *
from transformers import default_data_collator
from transformers import BertConfig, BertForQuestionAnswering, TrainingArguments, Trainer

dir_path = "/data/NFS/andy/course/ADL/hw2/"
data_path = dir_path + "/dataset/train.json"
context_path = dir_path + "/dataset/context.json"
weights_path = dir_path + "/qa_weigths/"

model_checkpoint = "bert-base-chinese"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
squad_v2 = False

In [2]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [3]:
data = load_json(data_path)
context = load_json(context_path)

for i in range(len(data)):
    data[i]["answers"] = {"answer_start":[data[i]["answers"][0]["start"]], "text": [data[i]["answers"][0]["text"]]}
    
length = int(len(data)*0.8)
train_data = {"id":[], "question":[], "context":[], "answers":[]}
val_data = {"id":[], "question":[], "context":[], "answers":[]}
for sub in data[:length]:
    train_data["id"].append(sub["id"])
    train_data["question"].append(sub["question"])
    train_data["context"].append(context[sub["relevant"]])
    train_data["answers"].append(sub["answers"])
for sub in data[length:]:
    val_data["id"].append(sub["id"])
    val_data["question"].append(sub["question"])
    val_data["context"].append(context[sub["relevant"]])
    val_data["answers"].append(sub["answers"])
    
datasets = DatasetDict({"train":Dataset.from_dict(train_data), "val":Dataset.from_dict(val_data)})

[*] Loading /data/NFS/andy/course/ADL/hw2//dataset/train.json...done
[*] Loading /data/NFS/andy/course/ADL/hw2//dataset/context.json...done


In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = tokenizer.padding_side == "right"

tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

In [5]:
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    num_attention_heads=4,
    num_hidden_layers=4,
    hidden_size=200,
)

model = BertForQuestionAnswering(config=config)

In [6]:
batch_size = 16
data_collator = default_data_collator

args = TrainingArguments(
    output_dir = "./qa_checkpoints",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps = 32,
)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [7]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,3.939926
1,No log,3.820301
2,No log,3.793323


TrainOutput(global_step=201, training_loss=4.011803470440765, metrics={'train_runtime': 945.4637, 'train_samples_per_second': 0.213, 'total_flos': 2363820532715520.0, 'epoch': 2.99})

In [8]:
validation_features = datasets["val"].map(prepare_validation_features, batched=True, 
                                          remove_columns=datasets["val"].column_names)

In [9]:
raw_predictions = trainer.predict(validation_features)

In [10]:
final_predictions = postprocess_qa_predictions(datasets["val"], validation_features, raw_predictions.predictions)

submission = {}
for i in range(len(datasets["val"])):
    submission[datasets["val"][i]["id"]] = final_predictions[datasets["val"][i]["id"]]

  0%|          | 9/4882 [00:00<01:00, 80.70it/s]

Post-processing 4882 example predictions split into 8639 features.


100%|██████████| 4882/4882 [01:06<00:00, 73.12it/s]


In [11]:
data = load_json(data_path)
length = int(len(data)*0.8)
data_val = data[length:]
answers = collect_answers(data_val)

[*] Loading /data/NFS/andy/course/ADL/hw2//dataset/train.json...done


In [12]:
tokenizer = Tokenizer()

In [13]:
result = compute_metrics(answers, submission, tokenizer)
pprint(result)

[*] Evaluating: 100%|██████████| 4882/4882 [01:01<00:00, 79.42it/s]

{'count': 4882, 'em': 0.035026628430970916, 'f1': 0.05885838279338571}


In [14]:
submission

{'6be639fee2db0a2e29f1e6ba9abca4ef': '2005年',
 '7fcee8b3933276c897257056501aedef': '俾斯麥在1870年',
 '47733bf9142a83834cc499f1541ce46c': '2001年',
 '1030a3c439891995ca54ccac44d2c034': '2010年',
 'c3cd01bb6ede5148da47e6495d35bd5c': '4000公尺。馬德拉的火山已經不活躍，最後一次噴發約在40萬年',
 'f3ab19aed28fb12bd905c1e95014c850': '戲曲作為文學',
 '85177707bc72f8344dcb6e6716b2584b': '60年',
 'da42a1fbfa1d5c37adc4c50fabcc8a9e': '1948年8月大韓民國宣布建國。同年',
 'a1084680fff0abb2e1a10bf053b2627c': '氫',
 'f81a11788ad84c1edf1a43545aa77330': '劉裕進封宋公，東西堂',
 '371e1353ccdf71f624dd17a4de0b621a': '2008年',
 'f0a5ff24f93f08c542527622296e932a': '1948年',
 'b9e4ba4bc309789c1c50006ff551c22f': '1993年',
 '1da84db36d5ce6a1a04df2e7c8530b11': '歐洲西北部在22,000年',
 'faee8d84485e4d5a11e6cd2a7565de8e': '18.6年',
 'f3e3bac8bb9d80263a0b211657d4231f': '四個較有代',
 '635ca81575ee296ea1477a81e4f00cdf': '巴泰勒米，台灣又譯聖巴瑟米',
 '5f00a80e4b14c0a5dc77d7caf2649589': '宋，合稱兩宋，國祚共319年',
 '239d594fd5fb13d4079d229cbf33e066': '106歲',
 '09d895941cab78c8b9cf3d2251429e2b': '1976年',
 'cb486ba2e20